In [3]:
import constant as c
import functions as f
import pandas as pd
import os
from os.path import join
from itertools import combinations
import numpy as np
from scipy.spatial.distance import euclidean, hamming



In [4]:
ant_dataframe = []

for filename in os.listdir(c.ant):
    tmp = pd.read_csv(join(c.ant, filename))
    ant_dataframe.append(tmp)

ant_dataframe = pd.concat(ant_dataframe)

In [5]:
ant_dataframe

,project,version,name,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.3,org.apache.tools.ant.taskdefs.ExecuteOn,11,4,2,14,42,29,2,...,1.000000,1,0.885057,0.232323,3,4,34.545455,3,1.2727,0
1,ant,1.3,org.apache.tools.ant.DefaultLogger,14,1,1,8,32,49,4,...,1.000000,0,0.000000,0.307692,0,0,16.857143,6,1.6429,2
2,ant,1.3,org.apache.tools.ant.taskdefs.TaskOutputStream,3,2,0,1,9,0,0,...,1.000000,1,0.714286,0.666667,1,1,17.333333,1,0.6667,0
3,ant,1.3,org.apache.tools.ant.taskdefs.Cvs,12,3,0,12,37,32,0,...,1.000000,1,0.770833,0.458333,0,0,24.083333,3,1.4167,0
4,ant,1.3,org.apache.tools.ant.taskdefs.Copyfile,6,3,0,4,21,1,0,...,1.000000,0,0.880952,0.416667,2,2,21.000000,1,0.8333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,ant,1.7,org.apache.tools.ant.taskdefs.Javadoc,92,3,0,34,261,3726,8,...,0.970588,11,0.291339,0.112476,2,2,28.021739,15,1.5543,4
741,ant,1.7,org.apache.tools.ant.types.selectors.BaseSelector,6,3,6,10,10,3,7,...,1.000000,0,0.857143,0.500000,0,0,6.500000,3,1.5000,0
742,ant,1.7,org.apache.tools.ant.types.resources.selectors...,7,3,5,9,26,0,5,...,1.000000,0,0.857143,0.314286,1,3,19.000000,3,1.5714,0
743,ant,1.7,org.apache.tools.ant.taskdefs.compilers.Gcj,5,2,0,8,34,8,1,...,1.000000,0,0.884615,1.000000,0,0,42.000000,11,3.4000,1


In [6]:
needed = ['name', 'wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom',
       'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic',
       'cbm', 'amc', 'max_cc', 'avg_cc', 'bug']
features = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom',
       'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic',
       'cbm', 'amc', 'max_cc', 'avg_cc', 'bug']

In [7]:
versions = ant_dataframe.version.unique()
names = ant_dataframe.name.unique()
versions

array([1.3, 1.4, 1.5, 1.6, 1.7])

In [8]:
for v in combinations(versions ,2):
    previous = ant_dataframe.loc[ant_dataframe.version == v[0]]
    current = ant_dataframe.loc[ant_dataframe.version == v[1]]
    previous = previous.sort_values('name').reset_index()
    current = current.sort_values('name').reset_index()
    previous = previous.drop(['index'], axis = 1)
    current = current.drop(['index'], axis = 1)
    modules_previous = previous.name.unique()
    modules_current = current.name.unique()
    D = f.create_D(current, previous, features, 'euclidean')
    print(D)
   
    break

[[ 852.83887089  622.12365795  595.26839356 ...  574.36722762
   448.64379945  631.8679798 ]
 [1415.36716108    6.4807407   192.37897729 ...   54.52576716
  1066.0483796    19.11017728]
 [1375.10757995  137.88281583   52.36410985 ...  138.84211074
  1018.32300851  147.40046063]
 ...
 [ 937.06592773  513.62325484  497.02763369 ...  466.28374848
   559.13649862  523.61136002]
 [1100.75894413  342.71645937  344.11501281 ...  294.63993035
   724.44022017  352.19489713]
 [1250.47081847  180.28335019  228.95982301 ...  130.96524626
   888.36364548  188.64378215]]


In [28]:
k = [2,2,3,4,5,2]
m = np.min(k)
print(m)

k = [1 if c == m else 0 for c in k]
print(k)

2
[1, 1, 0, 0, 0, 1]


In [60]:
D.shape

(125, 178)

In [47]:
Z = np.empty([len(previous), len(current)])

for index, d in enumerate(D):
    m = np.min(d)
    k = [1 if c == m else 0 for c in d]
    Z[index] = k

In [61]:
current

,project,version,name,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.4,org.apache.tools.ant.AntClassLoader,39,2,0,15,103,609,10,...,0.636364,1,0.500000,0.184211,1,5,33.743590,12,1.8462,0
1,ant,1.4,org.apache.tools.ant.BuildEvent,11,2,0,12,15,13,9,...,1.000000,3,0.200000,0.220779,0,0,7.272727,1,0.7273,0
2,ant,1.4,org.apache.tools.ant.BuildException,14,4,0,136,26,0,135,...,1.000000,1,0.650000,0.333333,1,2,9.785714,2,0.6429,0
3,ant,1.4,org.apache.tools.ant.BuildListener,7,1,0,11,7,21,10,...,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
4,ant,1.4,org.apache.tools.ant.BuildLogger,4,1,0,4,4,6,3,...,0.000000,0,0.000000,0.500000,0,0,0.000000,1,1.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,ant,1.4,org.apache.tools.zip.ZipEntry,23,2,0,5,68,181,2,...,0.875000,0,0.547619,0.140909,0,0,18.739130,5,1.4783,0
174,ant,1.4,org.apache.tools.zip.ZipExtraField,6,1,0,5,6,15,4,...,0.000000,0,0.000000,0.444444,0,0,0.000000,1,1.0000,0
175,ant,1.4,org.apache.tools.zip.ZipLong,7,1,0,2,8,0,2,...,1.000000,0,0.000000,0.342857,1,1,19.857143,4,1.0000,0
176,ant,1.4,org.apache.tools.zip.ZipOutputStream,17,4,0,7,81,40,4,...,0.904762,8,0.516129,0.250000,2,4,65.941176,2,0.9412,0


In [67]:
df = pd.DataFrame(data=Z.T)
dfd = pd.DataFrame(data=D)

In [66]:
df

,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
174,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
dfd

,0,1,2,3,4,5,6,7,8,9,...,168,169,170,171,172,173,174,175,176,177
0,852.838871,622.123658,595.268394,711.701084,716.314891,721.424934,436.609358,601.045972,657.290126,784.330848,...,444.729841,343.075666,341.582207,714.486484,641.656021,273.263023,713.288166,574.367228,448.643799,631.867980
1,1415.367161,6.480741,192.378977,91.494134,94.866471,100.970653,186.608634,25.161683,47.969638,1404.417831,...,179.894653,304.198619,283.937850,94.149205,28.480713,405.719347,92.144557,54.525767,1066.048380,19.110177
2,1375.107580,137.882816,52.364110,194.680159,202.925965,210.024562,187.581421,142.992752,169.799341,1354.034998,...,187.900507,292.556913,265.779356,204.022834,160.623427,384.822171,200.583978,138.842111,1018.323009,147.400461
3,1494.856149,91.581530,236.922697,7.071068,16.408839,26.495283,275.425505,113.228944,68.553710,1493.361285,...,269.400038,387.179071,374.248577,25.159491,70.609992,486.916431,6.504509,143.103941,1156.503903,85.546622
4,1503.890734,95.507316,241.691844,19.729420,1.414214,10.781929,280.705836,116.040514,67.694370,1498.241792,...,273.252778,394.945777,377.817964,10.062306,77.513844,495.372589,10.247101,144.676642,1160.079212,86.434386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,1488.564734,78.949812,232.401198,32.588341,22.366269,24.041631,263.080859,97.926034,51.477611,1479.441342,...,254.501534,378.994065,358.955193,17.000000,63.958745,480.057131,26.176278,125.345218,1140.735579,67.787031
121,729.890244,692.085653,669.472455,775.848348,783.501813,789.767907,508.977424,673.063202,729.189198,747.167673,...,523.197647,388.919871,433.682357,783.138779,706.414292,292.123998,778.636025,650.718967,462.354492,704.625808
122,937.065928,513.623255,497.027634,602.622383,607.496731,612.680949,327.888791,492.667806,548.954746,892.062077,...,336.097331,233.550631,236.005193,605.776316,532.306708,175.482578,604.246025,466.283748,559.136499,523.611360
123,1100.758944,342.716459,344.115013,432.688218,436.794752,441.831774,158.700024,321.464953,378.032687,1063.036693,...,165.673777,108.743157,66.620521,434.902652,362.616553,146.158769,433.954649,294.639930,724.440220,352.194897
